### We want to load Historical data of all Belgian GPs, along with driver info, team info, tire compound, weather, sector and lap times

In [1]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache("../f1_cache")

In [5]:
years = [2022, 2023, 2024]
race = "Belgian"
all_laps_rain = []
all_laps_dry = []

In [6]:
for year in years:
    try:
        session = fastf1.get_session(year, race, "Q")   # Get all details of the Qualifying sessions
        session.load()

        weather = session.weather_data
        avg_by_rain = weather.groupby(["Rainfall"]).mean()

        for drv in session.drivers:
            laps = session.laps.pick_driver(drv).pick_accurate()
            driver_best = laps[~laps['PitOutTime'].notna() & ~laps['PitInTime'].notna() & laps['IsPersonalBest'] == True]
            avg_sector_one = driver_best["Sector1Time"].dt.total_seconds().mean()
            avg_sector_two = driver_best["Sector2Time"].dt.total_seconds().mean()
            avg_sector_three = driver_best["Sector3Time"].dt.total_seconds().mean()
            avg_lap = driver_best['LapTime'].dt.total_seconds().mean()
            compound_counts = driver_best['Compound'].value_counts().to_dict()
            compound_used = max(compound_counts, key=compound_counts.get)

            team = session.get_driver(drv)['TeamName']

            all_laps_rain.append({
                    'Year': year,
                    'Driver': drv,
                    'Team': team,
                    'AvgLapTime': avg_lap,
                    'Compound': compound_used,
                    'AvgSectorOneTime': avg_sector_one,
                    'AvgSectorTwoTime': avg_sector_two,
                    'AvgSectorThreeTime': avg_sector_three,
                    'AirTemp': avg_by_rain.loc[True]['AirTemp'],
                    'TrackTemp': avg_by_rain.loc[True]['TrackTemp'],
                    'Humidity': avg_by_rain.loc[True]['Humidity'],
                    'WindSpeed': avg_by_rain.loc[True]['WindSpeed']
                })
            
            all_laps_dry.append({
                    'Year': year,
                    'Driver': drv,
                    'Team': team,
                    'AvgLapTime': avg_lap,
                    'Compound': compound_used,
                    'AvgSectorOneTime': avg_sector_one,
                    'AvgSectorTwoTime': avg_sector_two,
                    'AvgSectorThreeTime': avg_sector_three,
                    'AirTemp': avg_by_rain.loc[False]['AirTemp'],
                    'TrackTemp': avg_by_rain.loc[False]['TrackTemp'],
                    'Humidity': avg_by_rain.loc[False]['Humidity'],
                    'WindSpeed': avg_by_rain.loc[False]['WindSpeed']
                })
        
        
    except Exception as e:
        print(f"Failed for {year}: {e}")

core           INFO 	Loading data for Belgian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '11', '16', '31', '14', '44', '63', '23', '4', '3', '10', '24', '18', '47', '5', '6', '20', '22', '77']
/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and w

Failed for 2022: True


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '44', '55', '81', '4', '63', '14', '18', '22', '10', '20', '77', '31', '23', '24', '2', '3', '27']
/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be rem

In [7]:
all_data_rain = pd.DataFrame(all_laps_rain)
all_data_rain.dropna(inplace=True)
all_data_rain.head()

,Year,Driver,Team,AvgLapTime,Compound,AvgSectorOneTime,AvgSectorTwoTime,AvgSectorThreeTime,AirTemp,TrackTemp,Humidity,WindSpeed
0,2023,1,Red Bull Racing,115.268125,INTERMEDIATE,32.878500,51.533000,30.856625,18.0,21.6,93.0,0.6
1,2023,16,Ferrari,116.169375,INTERMEDIATE,32.846000,52.033875,31.289500,18.0,21.6,93.0,0.6
2,2023,11,Red Bull Racing,115.084375,INTERMEDIATE,32.909625,51.425250,30.749500,18.0,21.6,93.0,0.6
3,2023,44,Mercedes,115.394500,INTERMEDIATE,32.688000,51.871500,30.835000,18.0,21.6,93.0,0.6
4,2023,55,Ferrari,115.801444,INTERMEDIATE,32.767333,52.079778,30.954333,18.0,21.6,93.0,0.6


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [9]:
df_rain = all_data_rain.copy()

In [15]:
categories = ['Driver', 'Team', 'Compound']
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_rain[categories])

In [16]:
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categories))

In [17]:
numerical_cols = ['AvgSectorOneTime', 'AvgSectorTwoTime', 'AvgSectorThreeTime',
                  'AirTemp', 'TrackTemp', 'Humidity', 'WindSpeed']
X = pd.concat([encoded_df, df_rain[numerical_cols]], axis=1)
y = df_rain['AvgLapTime']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [20]:
model = RandomForestRegressor(n_estimators=100, random_state=7)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=7)

In [21]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [22]:
print(f"MAE: {mae:.3f} seconds")
print(f"R²: {r2:.3f}")

MAE: 0.399 seconds
R²: 0.974
